In [13]:
%pip install seaborn
%pip install matplotlib

You should consider upgrading via the '/Users/ame/.pyenv/versions/3.9.6/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Users/ame/.pyenv/versions/3.9.6/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [14]:
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization
from tensorflow.keras.layers import MaxPool2D, GlobalAvgPool2D
from tensorflow.keras.layers import Add, ReLU, Dense
from tensorflow.keras import Model
import matplotlib.pyplot as plt
%matplotlib inline 

In [15]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
gpus

if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)


In [16]:
#Conv-BatchNorm-ReLU block

def conv_batchnorm_relu(x, filters, kernel_size, strides=1):
    
    x = Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

In [17]:
#Identity block

def identity_block(tensor, filters):
    
    x = conv_batchnorm_relu(tensor, filters=filters, kernel_size=1, strides=1)
    x = conv_batchnorm_relu(x, filters=filters, kernel_size=3, strides=1)
    x = Conv2D(filters=4*filters, kernel_size=1, strides=1)(x)
    x = BatchNormalization()(x)
    
    x = Add()([tensor,x])    #skip connection
    x = ReLU()(x)
    
    return x

In [18]:
#Projection block

def projection_block(tensor, filters, strides):
    
    #left stream
    x = conv_batchnorm_relu(tensor, filters=filters, kernel_size=1, strides=strides)
    x = conv_batchnorm_relu(x, filters=filters, kernel_size=3, strides=1)
    x = Conv2D(filters=4*filters, kernel_size=1, strides=1)(x)
    x = BatchNormalization()(x)
    
    #right stream
    shortcut = Conv2D(filters=4*filters, kernel_size=1, strides=strides)(tensor)
    shortcut = BatchNormalization()(shortcut)
    
    x = Add()([shortcut,x])    #skip connection
    x = ReLU()(x)
    
    return x

In [19]:
#Resnet block

def resnet_block(x, filters, reps, strides):
    
    x = projection_block(x, filters, strides)
    for _ in range(reps-1):
        x = identity_block(x,filters)
        
    return x

In [20]:
#Model
# CONSTANTS
imagesize = 128
OUTPUT = 7
CLASSES = ['HeadShotHyalomma', 'HeadShotRhipicephalus',
        'CenterShotHyalomma', 'CenterShotRhipicephalus',
        'CornerShotHyalomma', 'CornerShotRhipicephalus',
        'Unidentified']

input = Input(shape=(imagesize, imagesize, 3))

x = conv_batchnorm_relu(input, filters=64, kernel_size=7, strides=2)
x = MaxPool2D(pool_size = 3, strides =2)(x)
x = resnet_block(x, filters=64, reps =3, strides=1)
x = resnet_block(x, filters=128, reps =4, strides=2)
x = resnet_block(x, filters=256, reps =6, strides=2)
x = resnet_block(x, filters=512, reps =3, strides=2)
x = GlobalAvgPool2D()(x)

output = Dense(OUTPUT, activation ='softmax')(x)

model = Model(inputs=input, outputs=output)
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 32, 32, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_53 (Conv2D)  │ (None, 16, 16,    │      9,472 │ input_layer_1[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 16, 16,    │        256 │ conv2d_53[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_49 (ReLU)     │ (None, 16, 16,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 7, 7, 64)  │          0 │ re_lu_49[0][0]    │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_54 (Conv2D)  │ (None, 7, 7, 64)  │      4,160 │ max_pooling2d_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 7, 7, 64)  │        256 │ conv2d_54[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_50 (ReLU)     │ (None, 7, 7, 64)  │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_55 (Conv2D)  │ (None, 7, 7, 64)  │     36,928 │ re_lu_50[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 7, 7, 64)  │        256 │ conv2d_55[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_51 (ReLU)     │ (None, 7, 7, 64)  │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_57 (Conv2D)  │ (None, 7, 7, 256) │     16,640 │ max_pooling2d_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_56 (Conv2D)  │ (None, 7, 7, 256) │     16,640 │ re_lu_51[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 7, 7, 256) │      1,024 │ conv2d_57[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 7, 7, 256) │      1,024 │ conv2d_56[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_16 (Add)        │ (None, 7, 7, 256) │          0 │ batch_normalizat… │
│                     │                   │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_52 (ReLU)     │ (None, 7, 7, 256) │          0 │ add_16[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_58 (Conv2D)  │ (None, 7, 7, 64)  │     16,448 │ re_lu_52[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 7, 7, 64)  │        256 │ conv2d_58[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 23,593,859 (90.00 MB)

 Trainable params: 23,540,739 (89.80 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [26]:
RESNET50_POOLING_AVERAGE = 'avg'
DENSE_LAYER_ACTIVATION = 'softmax'
OBJECTIVE_FUNCTION = 'categorical_crossentropy'
LOSS_METRICS = ['accuracy']
CLASS_MODE = 'categorical'
NUM_EPOCHS = 30

In [27]:
from tensorflow.keras import optimizers

sgd = optimizers.Adam(learning_rate= 0.0001)
model.compile(optimizer = sgd, loss = OBJECTIVE_FUNCTION, metrics = LOSS_METRICS)

In [50]:
from keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# preprocessing_function is applied on each image but only after re-sizing & augmentation (resize => augment => pre-process)
# Each of the keras.application.resnet* preprocess_input MOSTLY mean BATCH NORMALIZATION (applied on each batch) stabilize the inputs to nonlinear activation functions
# Batch Normalization helps in faster convergence
# data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)
data_generator = ImageDataGenerator(rescale=1.0/255.0)

# flow_From_directory generates batches of augmented data (where augmentation can be color conversion, etc)
# Both train & valid folders must have NUM_CLASSES sub-folders
train_generator = data_generator.flow_from_directory(
        'datasetB/train',
        target_size=(imagesize, imagesize),
        # batch_size=BATCH_SIZE_TRAINING,
        class_mode=CLASS_MODE,
        classes=CLASSES)

validation_generator = data_generator.flow_from_directory(
        'datasetB/dev',
        target_size=(imagesize, imagesize),
        # batch_size=BATCH_SIZE_VALIDATION,
        class_mode=CLASS_MODE,
        classes=CLASSES
) 

Found 12146 images belonging to 3 classes.
Found 9717 images belonging to 3 classes.


In [29]:
fit_history = model.fit(
        train_generator,
        epochs = NUM_EPOCHS,
        validation_data=validation_generator,     
)

Epoch 1/3


/Users/ame/.pyenv/versions/3.9.6/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


380/380 ━━━━━━━━━━━━━━━━━━━━ 416s 1s/step - accuracy: 0.6513 - loss: 0.8429 - val_accuracy: 0.6812 - val_loss: 0.8438
Epoch 2/3
380/380 ━━━━━━━━━━━━━━━━━━━━ 445s 1s/step - accuracy: 0.7795 - loss: 0.5331 - val_accuracy: 0.8139 - val_loss: 0.4515
Epoch 3/3
380/380 ━━━━━━━━━━━━━━━━━━━━ 400s 1s/step - accuracy: 0.8470 - loss: 0.3818 - val_accuracy: 0.8539 - val_loss: 0.3619


In [30]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, classification_report, roc_auc_score, roc_curve

model_type = "RESNET50"
variant = "VARB"
count = "00k"
epochs = NUM_EPOCHS
result_folder_name = "resnet"
results_folder = f'results/b/{result_folder_name}/{str(NUM_EPOCHS)}'


# Create results folder if it doesn't exist
if not os.path.exists(results_folder):
    os.makedirs(results_folder)

In [32]:
plt.figure(1, figsize = (15,8)) 
    
# plt.subplot(221)  
plt.plot(fit_history.history['accuracy'])  
plt.plot(fit_history.history['val_accuracy'])  
plt.title('model accuracy')  
plt.ylabel('accuracy')  
plt.xlabel('epoch')  
plt.legend(['train', 'valid']) 
plt.savefig(os.path.join(results_folder, 'model-accuracy.png'))
plt.close()

# plt.subplot(222)  
plt.plot(fit_history.history['loss'])  
plt.plot(fit_history.history['val_loss'])  
plt.title('model loss')  
plt.ylabel('loss')  
plt.xlabel('epoch')  
plt.legend(['train', 'valid']) 
plt.savefig(os.path.join(results_folder, 'model-loss.png'))
plt.close()

In [52]:
# Make predictions on the test set
training_loss, training_accuracy = model.evaluate(train_generator)
print("Training Loss:", training_loss)
print("Training Accuracy:", training_accuracy)

validation_loss, validation_accuracy = model.evaluate(train_generator)
print("Validation Loss:", validation_loss)
print("Validation Accuracy:", validation_accuracy)

380/380 ━━━━━━━━━━━━━━━━━━━━ 117s 306ms/step - accuracy: 0.9083 - loss: 0.2448
Training Loss: 0.2426428347826004
Training Accuracy: 0.905894935131073
380/380 ━━━━━━━━━━━━━━━━━━━━ 117s 308ms/step - accuracy: 0.9078 - loss: 0.2378
Validation Loss: 0.24264273047447205
Validation Accuracy: 0.905894935131073


In [53]:
# Save ROC AUC
with open(os.path.join(results_folder, 'training_validation.txt'), 'w') as f:
    f.write(f'training_accuracy: {training_accuracy}\n')
    f.write(f'training_loss: {training_loss}\n')
    f.write(f'validation_accuracy: {validation_accuracy}\n')
    f.write(f'validation_loss: {validation_loss}\n')

In [34]:
# # Define the model loading function
# def load_my_model(model_path):
#     return load_model(model_path)

# Paths and constants
# model_path = 'models/' + name
data_folder = 'datasetB/test'

# Load the model
# model = load_my_model(model_path)

# Data generator for the test set
testingData = ImageDataGenerator(rescale=1.0/255.0)

testdata = testingData.flow_from_directory(
    data_folder,
    target_size=(imagesize, imagesize),  # Change this to the input size of your model
    class_mode='categorical',  # or 'binary' depending on your model
    shuffle=False,
    classes=CLASSES
)



Found 3947 images belonging to 3 classes.


In [54]:
# Make predictions on the test set
test_loss, test_accuracy = model.evaluate(testdata)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

with open(os.path.join(results_folder, 'test.txt'), 'w') as f:
    f.write(f'test_accuracy: {test_accuracy}\n')
    f.write(f'test_loss: {test_loss}\n')

y_true = testdata.classes
y_pred_probs = model.predict(testdata, steps=None)

print("testdata.samples:", testdata.samples)
print("len(y_true):", len(y_true))

y_pred = np.argmax(y_pred_probs, axis=1)
print("y_pred: ", len(y_pred))

124/124 ━━━━━━━━━━━━━━━━━━━━ 22s 172ms/step - accuracy: 0.8377 - loss: 0.4218
Test Loss: 0.36693865060806274
Test Accuracy: 0.8571066856384277
124/124 ━━━━━━━━━━━━━━━━━━━━ 21s 171ms/step
testdata.samples: 3947
len(y_true): 3947
y_pred:  3947


In [55]:
# Calculate confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)

# Calculate accuracy
accuracy = accuracy_score(y_true, y_pred)

# Calculate precision
precision = precision_score(y_true, y_pred, average='weighted')

# Calculate sensitivity (recall)
sensitivity = recall_score(y_true, y_pred, average='weighted')

# Calculate F1 score
f1 = precision_score(y_true, y_pred, average='weighted')

# Print the results
print("Confusion Matrix:")
print(conf_matrix)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Sensitivity (Recall):", sensitivity)
print("F1 Score:", f1)

# Calculate classification report
report = classification_report(y_true, y_pred, target_names=CLASSES, output_dict=True)

# Save classification report
report_df = pd.DataFrame(report).transpose()
report_df.to_csv(os.path.join(results_folder, 'classification_report.csv'))

# Calculate ROC AUC score
roc_auc = roc_auc_score(y_true, y_pred_probs, multi_class='ovo')

# Save ROC AUC
with open(os.path.join(results_folder, 'roc_auc.txt'), 'w') as f:
    f.write(f'ROC AUC: {roc_auc}\n')

# Calculate sensitivity and specificity for each class
sensitivity_per_class = np.diag(conf_matrix) / np.sum(conf_matrix, axis=1)

# Calculate true negatives, false positives, and specificity for each class
tn = []
fp = []
specificity_per_class = []

for i in range(len(CLASSES)):
    true_negative = np.sum(conf_matrix) - (np.sum(conf_matrix[i, :]) + np.sum(conf_matrix[:, i]) - conf_matrix[i, i])
    false_positive = np.sum(conf_matrix[:, i]) - conf_matrix[i, i]
    tn.append(true_negative)
    fp.append(false_positive)
    specificity_per_class.append(true_negative / (true_negative + false_positive))

# Save confusion matrix, sensitivity, specificity, and recall
conf_matrix_df = pd.DataFrame(conf_matrix, index=CLASSES, columns=CLASSES)
conf_matrix_df.to_csv(os.path.join(results_folder, 'confusion_matrix.csv'))

sensitivity_df = pd.DataFrame([sensitivity_per_class], index=['Sensitivity'], columns=CLASSES)
sensitivity_df.to_csv(os.path.join(results_folder, 'sensitivity.csv'))

specificity_df = pd.DataFrame([specificity_per_class], index=['Specificity'], columns=CLASSES)
specificity_df.to_csv(os.path.join(results_folder, 'specificity.csv'))

recall_df = pd.DataFrame([sensitivity_per_class], index=['Recall'], columns=CLASSES)
recall_df.to_csv(os.path.join(results_folder, 'recall.csv'))

# Plot and save confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=CLASSES, yticklabels=CLASSES)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.savefig(os.path.join(results_folder, 'confusion_matrix.png'))
plt.close()

# Plot and save ROC curve
plt.figure()
for i, class_name in enumerate(CLASSES):
    fpr, tpr, _ = roc_curve(y_true == i, y_pred_probs[:, i])
    plt.plot(fpr, tpr, label=f'{class_name} (AUC: {roc_auc_score(y_true == i, y_pred_probs[:, i]):.2f})')

plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.savefig(os.path.join(results_folder, 'roc_curve.png'))
plt.close()

print("Results and plots saved to", results_folder)

Confusion Matrix:
[[ 348  121    9]
 [  23 1102   66]
 [  86  259 1933]]
Accuracy: 0.8571066632885735
Precision: 0.8721869699836771
Sensitivity (Recall): 0.8571066632885735
F1 Score: 0.8721869699836771
Results and plots saved to results/resnet/3


In [66]:
from datetime import date
import math

accuracy = accuracy * 100 
accuracy = math.floor(accuracy * 100) / 100

today = date.today()

# dd/mm/YY
date = "-".join(today.strftime("%d/%m/%Y").split("/"))
print(date)

name = f"{model_type}_{variant}{count}_{imagesize}x{imagesize}_EP{epochs}_ACCU{accuracy}_{date}.keras"

model.save('models/' + name)

23-06-2024
